# UES Index NestedLoopJoin workload

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import pandas as pd

from transform import mosp
from postgres import explain

## Data loading and preparation

In [3]:
df_full = pd.read_csv("workloads/job-ues-results-idxnlj.csv")

The full result file contains data for numerous repetitions. However, for the analysis we should only take a look at a single run. Therefore, we will select the fastest runs per setting as representative for each workload.

In [30]:
experiment_rt = df_full.groupby(["workload", "run"]).rt_total.sum()
best_runs = experiment_rt.loc[experiment_rt.groupby("workload").idxmin()].reset_index().drop(columns="rt_total")
best_runs

,workload,run
0,ues,4
1,ues_idxnlj,4
2,ues_idxnlj_fullnlj,5


The representative experiments will be selected by executing a semi-join of the full result set with the best runs.

In [32]:
df_repr = df_full.merge(best_runs[(best_runs.workload == "ues") | (best_runs.workload == "ues_idxnlj")], how="inner", on=["workload", "run"])
df_repr

,label,query,query_result,rt_total,run,workload,query_hint
0,1a,select count(*) from movie_companies as mc j...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.792273,4,ues,NaN
1,1b,select count(*) from movie_info_idx as mi_idx...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.553238,4,ues,NaN
2,1c,select count(*) from movie_companies as mc j...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.636802,4,ues,NaN
3,1d,select count(*) from movie_info_idx as mi_idx...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.619424,4,ues,NaN
4,2a,select count(*) from movie_keyword as mk joi...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",1.095501,4,ues,NaN
...,...,...,...,...,...,...,...
221,32a,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.011646,4,ues_idxnlj,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/
222,32b,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.252471,4,ues_idxnlj,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/
223,33a,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",1.903043,4,ues_idxnlj,/*+\nIndexOnlyScan(mi_idx1)\nIndexOnlyScan(mi_...
224,33b,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.692811,4,ues_idxnlj,/*+\nIndexOnlyScan(mi_idx1)\nIndexOnlyScan(mi_...


An alternative selection of representative queries: Instead of choosing all queries from one single workload, queries are selected across workloads. A query is choosen, if it has the minimum runtime in all repetitions. This improves the overall (aggregated) workload runtime, since the optimal queries are selected instead of the optimal workload. However, the resulting workloads are purely virtual and "stitched together".

In [44]:
best_query_runs = df_full.groupby(["workload", "label"])["rt_total"].idxmin().to_frame().reset_index()
df_repr = df_full.loc[best_query_runs[(best_query_runs.workload == "ues") | (best_query_runs.workload == "ues_idxnlj")].rt_total].reset_index(drop=True)
df_repr

,label,query,query_result,rt_total,run,workload,query_hint
0,10a,select count(*) from cast_info as ci join ro...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",3.143667,4,ues,NaN
1,10b,select count(*) from cast_info as ci join ro...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",3.004887,1,ues,NaN
2,10c,select count(*) from cast_info as ci join ro...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",4.079468,1,ues,NaN
3,11a,select count(*) from movie_link as ml join l...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.101837,2,ues,NaN
4,11b,select count(*) from movie_companies as mc j...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.229032,1,ues,NaN
...,...,...,...,...,...,...,...
221,8d,SELECT COUNT(*) FROM aka_name AS an1 JOIN name...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",2.588374,2,ues_idxnlj,/*+\nIndexOnlyScan(ci)\nNestLoop(ci rt)\n*/
222,9a,SELECT COUNT(*) FROM aka_name AS an JOIN name ...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",3.397697,4,ues_idxnlj,NaN
223,9b,SELECT COUNT(*) FROM aka_name AS an JOIN name ...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",2.674708,1,ues_idxnlj,NaN
224,9c,SELECT COUNT(*) FROM aka_name AS an JOIN name ...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",4.061652,1,ues_idxnlj,NaN


Now we reduce the representative workload further, to only include those queries were hints have actually been applied.

In [45]:
df_hinted = df_repr[(df_repr.workload == "ues_idxnlj") & ~df_repr.query_hint.isna()]
df_orig = df_repr[(df_repr.workload == "ues") & df_repr.label.isin(df_hinted.label)]

In [46]:
df = pd.merge(df_orig.drop(columns=["run", "workload", "query_hint"]),
              df_hinted.drop(columns=["query", "run", "workload"]),
              on="label", suffixes=("_orig", "_hinted")).copy()
df["runtime_diff"] = df.rt_total_orig - df.rt_total_hinted


In [47]:
df["query"] = df["query"].apply(mosp.MospQuery.parse)
df["query_result_orig"] = df["query_result_orig"].apply(json.loads)
df["query_result_hinted"] = df["query_result_hinted"].apply(json.loads)
df["explain_hinted"] = df.apply(lambda row: explain.parse_explain_analyze(row["query"], row["query_result_hinted"], with_subqueries=True), axis="columns")

/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/postgres/explain.py:408: UserWarning: Unknown node type: Aggregate
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/postgres/explain.py:408: UserWarning: Unknown node type: Gather
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/postgres/explain.py:408: UserWarning: Unknown node type: Hash
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/postgres/explain.py:408: UserWarning: Unknown node type: Sort
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/postgres/explain.py:408: UserWarning: Unknown node type: Materialize
  warnings.warn("Unknown node type: {}".format(node_type))
/home/strix/Projekte/DiplomArbeit-QueryOpt/ues/postgres/explain.py:408: UserWarning: Unknown node type: Memoize
  warnings.warn("Unknown node type

In [48]:
df_sqs = df.copy()
df_sqs["subquery"] = df_sqs["query"].apply(lambda q: [sq.subquery for sq in q.subqueries()])
df_sqs = df_sqs.explode("subquery")
df_sqs["n_subquery_joins"] = df_sqs.subquery.apply(lambda sq: len(sq.joins()))
#df_sqs[df_sqs.n_subquery_joins > 1]

In [49]:
df["hinted_subquery"] = df.explain_hinted.apply(lambda e: e.extract_subqueries())
df = df.explode("hinted_subquery")

In [50]:
df["subquery_rows"] = df.hinted_subquery.apply(lambda sq: sq.proc_rows)
df["subquery_joins"] = df.hinted_subquery.apply(lambda sq: sq.depth()-1)
df["subquery_pk_rows"] = df.hinted_subquery.apply(lambda sq: sq.leaf_join().left.incoming_rows())
df["subquery_fk_rows"] = df.hinted_subquery.apply(lambda sq: sq.leaf_join().right.incoming_rows())
df

,label,query,query_result_orig,rt_total_orig,query_result_hinted,rt_total_hinted,query_hint,runtime_diff,explain_hinted,hinted_subquery,subquery_rows,subquery_joins,subquery_pk_rows,subquery_fk_rows
0,11a,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.101837,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.126027,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/,-0.024190,Hash Join (mk.movie_id = ml.movie_id) <- [~Nes...,~Nested Loop~ (mk.keyword_id = k.id) <- [~Seq ...,0,1,0,0
1,11b,SELECT COUNT(*) FROM movie_companies AS mc JOI...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.229032,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.328939,/*+\nIndexOnlyScan(ml)\nIndexOnlyScan(mk)\nNes...,-0.099907,Hash Join (mk.movie_id = t.id) <- [~Nested Loo...,~Nested Loop~ (mk.keyword_id = k.id) <- [~Seq ...,0,1,0,0
1,11b,SELECT COUNT(*) FROM movie_companies AS mc JOI...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.229032,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.328939,/*+\nIndexOnlyScan(ml)\nIndexOnlyScan(mk)\nNes...,-0.099907,Hash Join (mk.movie_id = t.id) <- [~Nested Loo...,~Nested Loop~ (ml.link_type_id = lt.id) <- [~S...,0,1,0,0
2,11c,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.025264,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.025669,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/,-0.000405,Merge Join (ml.movie_id = mk.movie_id) <- [Has...,~Nested Loop~ (k.id = mk.keyword_id) <- [~Inde...,0,1,0,0
3,11d,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.528066,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.228951,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/,0.299115,Merge Join (ml.movie_id = mk.movie_id) <- [Has...,Nested Loop (k.id = mk.keyword_id) <- [Index S...,313,1,94462,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,6e,SELECT COUNT(*) FROM movie_keyword AS mk JOIN ...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.863914,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.867937,/*+\nIndexOnlyScan(ci)\nNestLoop(ci n)\n*/,-0.004023,Hash Join (ci.movie_id = t.id) <- [Nested Loop...,Nested Loop (ci.person_id = n.id) <- [Seq Scan...,0,1,2126580,0
75,7a,SELECT COUNT(*) FROM person_info AS pi JOIN in...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.156443,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.152976,/*+\nIndexOnlyScan(ml)\nNestLoop(ml lt)\n*/,0.003467,Hash Join (t.id = ml.linked_movie_id) <- [Hash...,Nested Loop (ml.link_type_id = lt.id) <- [Seq ...,5186,1,18,5186
76,7b,SELECT COUNT(*) FROM person_info AS pi JOIN in...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.154768,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",0.150341,/*+\nIndexOnlyScan(ml)\nNestLoop(ml lt)\n*/,0.004427,Hash Join (t.id = ml.linked_movie_id) <- [Hash...,Nested Loop (ml.link_type_id = lt.id) <- [Seq ...,5186,1,18,5186
77,8c,SELECT COUNT(*) FROM aka_name AS a1 JOIN name ...,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",8.530332,"[{'Plan': {'Node Type': 'Aggregate', 'Strategy...",7.203760,/*+\nIndexOnlyScan(ci)\nNestLoop(ci rt)\n*/,1.326572,Hash Join (mc.company_id = cn.id) <- [Hash Joi...,Hash Join (t.id = ci.movie_id) <- [Seq Scan ::...,1725212,2,12,5175636


In [51]:
df_export = df.copy()
df_export.query_result_orig = df_export.query_result_orig.apply(json.dumps)
df_export.query_result_hinted = df_export.query_result_hinted.apply(json.dumps)

#df_export.to_csv("workloads/job-ues-eval-idxnlj.csv", index=False)
df_export.to_csv("workloads/job-ues-eval-idxnlj-queryopt.csv", index=False)
df_export

,label,query,query_result_orig,rt_total_orig,query_result_hinted,rt_total_hinted,query_hint,runtime_diff,explain_hinted,hinted_subquery,subquery_rows,subquery_joins,subquery_pk_rows,subquery_fk_rows
0,11a,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.101837,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.126027,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/,-0.024190,Hash Join (mk.movie_id = ml.movie_id) <- [~Nes...,~Nested Loop~ (mk.keyword_id = k.id) <- [~Seq ...,0,1,0,0
1,11b,SELECT COUNT(*) FROM movie_companies AS mc JOI...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.229032,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.328939,/*+\nIndexOnlyScan(ml)\nIndexOnlyScan(mk)\nNes...,-0.099907,Hash Join (mk.movie_id = t.id) <- [~Nested Loo...,~Nested Loop~ (mk.keyword_id = k.id) <- [~Seq ...,0,1,0,0
1,11b,SELECT COUNT(*) FROM movie_companies AS mc JOI...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.229032,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.328939,/*+\nIndexOnlyScan(ml)\nIndexOnlyScan(mk)\nNes...,-0.099907,Hash Join (mk.movie_id = t.id) <- [~Nested Loo...,~Nested Loop~ (ml.link_type_id = lt.id) <- [~S...,0,1,0,0
2,11c,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.025264,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.025669,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/,-0.000405,Merge Join (ml.movie_id = mk.movie_id) <- [Has...,~Nested Loop~ (k.id = mk.keyword_id) <- [~Inde...,0,1,0,0
3,11d,SELECT COUNT(*) FROM movie_link AS ml JOIN lin...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.528066,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.228951,/*+\nIndexOnlyScan(mk)\nNestLoop(mk k)\n*/,0.299115,Merge Join (ml.movie_id = mk.movie_id) <- [Has...,Nested Loop (k.id = mk.keyword_id) <- [Index S...,313,1,94462,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,6e,SELECT COUNT(*) FROM movie_keyword AS mk JOIN ...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.863914,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.867937,/*+\nIndexOnlyScan(ci)\nNestLoop(ci n)\n*/,-0.004023,Hash Join (ci.movie_id = t.id) <- [Nested Loop...,Nested Loop (ci.person_id = n.id) <- [Seq Scan...,0,1,2126580,0
75,7a,SELECT COUNT(*) FROM person_info AS pi JOIN in...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.156443,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.152976,/*+\nIndexOnlyScan(ml)\nNestLoop(ml lt)\n*/,0.003467,Hash Join (t.id = ml.linked_movie_id) <- [Hash...,Nested Loop (ml.link_type_id = lt.id) <- [Seq ...,5186,1,18,5186
76,7b,SELECT COUNT(*) FROM person_info AS pi JOIN in...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.154768,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",0.150341,/*+\nIndexOnlyScan(ml)\nNestLoop(ml lt)\n*/,0.004427,Hash Join (t.id = ml.linked_movie_id) <- [Hash...,Nested Loop (ml.link_type_id = lt.id) <- [Seq ...,5186,1,18,5186
77,8c,SELECT COUNT(*) FROM aka_name AS a1 JOIN name ...,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",8.530332,"[{""Plan"": {""Node Type"": ""Aggregate"", ""Strategy...",7.203760,/*+\nIndexOnlyScan(ci)\nNestLoop(ci rt)\n*/,1.326572,Hash Join (mc.company_id = cn.id) <- [Hash Joi...,Hash Join (t.id = ci.movie_id) <- [Seq Scan ::...,1725212,2,12,5175636


In [263]:
r = df.loc[df.label == "2c"].iloc[0]

In [264]:
print(r.hinted_subquery.pretty_print(include_filter=True))

[SQ] Nested Loop (mc.company_id = cn.id)
  <- Seq Scan :: company_name (((country_code)::text = '[sm]'::text))
  <- Index Scan :: movie_companies (company_id_movie_companies)



In [265]:
print(r.hinted_subquery.left.inspect_node())

Seq Scan :: company_name cn
-----------------------------------------------
  Join  : /
  Filter: ((country_code)::text = '[sm]'::text)
-----------------------------------------------
  Incoming Rows: 120711
  Filtered Rows: 120710
  Outgoing Rows: 1


In [266]:
print(r.hinted_subquery.right.inspect_node())

Index Scan :: movie_companies mc (company_id_movie_companies)
-------------------------------------------------------------
  Join  : /
  Filter: /
-------------------------------------------------------------
  Incoming Rows: 4
  Filtered Rows: /
  Outgoing Rows: 4


In [267]:
print(r.hinted_subquery.inspect_node())

Nested Loop
---------------------------------
  Join  : (mc.company_id = cn.id)
  Filter: /
---------------------------------
  Incoming Rows (left) : 1
  Incoming Rows (right): 4
  Filtered Rows: /
  Outgoing Rows: 2


In [220]:
r.query_result_hinted

[{'Plan': {'Node Type': 'Aggregate',
   'Strategy': 'Plain',
   'Partial Mode': 'Simple',
   'Parallel Aware': False,
   'Async Capable': False,
   'Startup Cost': 194827.21,
   'Total Cost': 194827.22,
   'Plan Rows': 1,
   'Plan Width': 8,
   'Actual Startup Time': 754.956,
   'Actual Total Time': 756.298,
   'Actual Rows': 1,
   'Actual Loops': 1,
   'Plans': [{'Node Type': 'Gather',
     'Parent Relationship': 'Outer',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 179249.23,
     'Total Cost': 194827.21,
     'Plan Rows': 1,
     'Plan Width': 0,
     'Actual Startup Time': 754.952,
     'Actual Total Time': 756.293,
     'Actual Rows': 0,
     'Actual Loops': 1,
     'Workers Planned': 2,
     'Workers Launched': 2,
     'Single Copy': False,
     'Plans': [{'Node Type': 'Hash Join',
       'Parent Relationship': 'Outer',
       'Parallel Aware': True,
       'Async Capable': False,
       'Join Type': 'Inner',
       'Startup Cost': 178249.23,
  

In [184]:
print(df.iloc[0].query_hint)

/*+
IndexOnlyScan(mi_idx)
NestLoop(mi_idx it)
*/


In [185]:
df.iloc[0].query_result_hinted

[{'Plan': {'Node Type': 'Aggregate',
   'Strategy': 'Plain',
   'Partial Mode': 'Simple',
   'Parallel Aware': False,
   'Async Capable': False,
   'Startup Cost': 234435.63,
   'Total Cost': 234435.64,
   'Plan Rows': 1,
   'Plan Width': 8,
   'Actual Startup Time': 672.034,
   'Actual Total Time': 672.831,
   'Actual Rows': 1,
   'Actual Loops': 1,
   'Plans': [{'Node Type': 'Gather',
     'Parent Relationship': 'Outer',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 139423.92,
     'Total Cost': 234435.62,
     'Plan Rows': 1,
     'Plan Width': 0,
     'Actual Startup Time': 355.504,
     'Actual Total Time': 672.79,
     'Actual Rows': 159,
     'Actual Loops': 1,
     'Workers Planned': 2,
     'Workers Launched': 2,
     'Single Copy': False,
     'Plans': [{'Node Type': 'Hash Join',
       'Parent Relationship': 'Outer',
       'Parallel Aware': False,
       'Async Capable': False,
       'Join Type': 'Inner',
       'Startup Cost': 138423.92,


## Minimalistic workload analysis

Runtime of the fastest repetitions per workload:

In [13]:
df_full.groupby(["workload", "run"])["rt_total"].sum().groupby("workload").min()

workload
ues                   265.445545
ues_idxnlj            260.734020
ues_idxnlj_fullnlj    270.619806
Name: rt_total, dtype: float64

Aggregated runtime of the fastests execution of each query per workload:

In [14]:
df_full.groupby(["workload", "label"])["rt_total"].min().groupby("workload").sum()

workload
ues                   262.580479
ues_idxnlj            255.313314
ues_idxnlj_fullnlj    267.134480
Name: rt_total, dtype: float64

In [15]:
df_full.groupby(["workload", "label"])["rt_total"].idxmin()

workload            label
ues                 10a      373
                    10b       35
                    10c       36
                    11a      150
                    11b       38
                            ... 
ues_idxnlj_fullnlj  8d       594
                    9a       821
                    9b       935
                    9c       710
                    9d       824
Name: rt_total, Length: 339, dtype: int64

In [26]:
df_full.groupby(["workload", "label"])["rt_total"].idxmin().to_frame().reset_index()

,workload,label,rt_total
0,ues,10a,373
1,ues,10b,35
2,ues,10c,36
3,ues,11a,150
4,ues,11b,38
...,...,...,...
334,ues_idxnlj_fullnlj,8d,594
335,ues_idxnlj_fullnlj,9a,821
336,ues_idxnlj_fullnlj,9b,935
337,ues_idxnlj_fullnlj,9c,710
